In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# #이후 런타임 다시 시작

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 1s (9,146 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 155222 files and d

In [ ]:
import warnings   
warnings.simplefilter(action='ignore', category=FutureWarning)

import matplotlib.pyplot as plt
import seaborn as sns

plt.rc('font', family='NanumBarunGothic') 
plt.rcParams["font.size"] = 12
plt.rcParams['xtick.labelsize'] = 12.
plt.rcParams['ytick.labelsize'] = 12.
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
import numpy as np
import pandas as pd
import re


from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import  accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import chi2

from ast import literal_eval
import statsmodels.api as sm
from sklearn.datasets import make_blobs
from statsmodels.formula.api import logit

In [ ]:
cd /content/drive/MyDrive/UBION_final_project/data

/content/drive/.shortcut-targets-by-id/1cbruNkx_Sh7hGut3yOs6kDVmEN0tQAuf/UBION_final_project/data


# Excute_model

In [ ]:
def change_cutoff(result_df):
    value_00 = result_df[(result_df['Actual']==0) & (result_df['prediction']== 0)]['부도_prob'].max()
    value_01 = result_df[(result_df['Actual']==0) & (result_df['prediction']== 1)]['부도_prob'].min()
    value_10 = result_df[(result_df['Actual']==1) & (result_df['prediction']== 0)]['부도_prob'].max()
    value_11 = result_df[(result_df['Actual']==1) & (result_df['prediction']== 1)]['부도_prob'].min()

    if value_10 < value_00:
        cutoff = value_11
        result_df.loc[(result_df['Actual']==0) & (result_df['prediction']== 1) & (result_df['부도_prob']<cutoff) , 'prediction'] = 0
    
    if value_11 > value_01:
        cutoff= value_00
        result_df.loc[(result_df['Actual']==1) & (result_df['prediction']== 0) & (result_df['부도_prob']>cutoff) , 'prediction'] = 1

    return result_df

In [ ]:
def Execute_optimal_lg_sm(X_train, y_train, X_test, y_test):

    logit = sm.Logit(y_train, X_train)
    result = logit.fit_regularized(alpha = 100, trim_mode='auto', method = 'l1_cvxopt_cp')
    print(result.summary())
    return result

In [ ]:
def Execute_optimal_lg(X_train, y_train, X_test, y_test, hyp):

    params = hyp
    lr = LogisticRegression(**params, random_state=0, max_iter=10000).fit(X_train, y_train)
    prediction = lr.predict(X_test)
    prediction_prob = lr.predict_proba(X_test) 

    y_test = pd.Series(y_test.values)
    y_test = y_test.to_frame()

    result_df = pd.DataFrame(prediction_prob, columns=['정상_prob','부도_prob'])
    result_df['prediction'] = prediction
    
    result_df['Actual'] = y_test

    fper, tper, thresholds = roc_curve(result_df['Actual'], result_df['부도_prob'])
    tmp_proba = [fper, tper]
    

    return tmp_proba

In [ ]:
def Execute_optimal_svm(X_train, y_train, X_test, y_test, hyp):
    params = hyp
    svm = SVC(**params, probability= True).fit(X_train, y_train)

    prediction = svm.predict(X_test)
    prediction_prob =svm.predict_proba(X_test)

    y_test = pd.Series(y_test.values)
    y_test = y_test.to_frame()

    result_df = pd.DataFrame(prediction_prob, columns=['정상_prob','부도_prob'])
    result_df['prediction'] = prediction
    result_df['Actual'] = y_test


    return_df = change_cutoff(result_df)

    fper, tper, thresholds = roc_curve(result_df['Actual'], result_df['부도_prob'])
    tmp_proba = [fper, tper]
    
    return tmp_proba

In [ ]:
def Execute_optimal_pr(X_train, y_train, X_test, y_test, hyp):
    dt2 = DecisionTreeClassifier(max_depth=2)
    dt2.fit(X_train, y_train)
    prediction = dt2.predict(X_test)
    prediction_prob =dt2.predict_proba(X_test)

    y_test = pd.Series(y_test.values)
    y_test = y_test.to_frame()

    result_df = pd.DataFrame(prediction_prob, columns=['정상_prob','부도_prob'])
    result_df['prediction'] = prediction
    result_df['Actual'] = y_test


    return_df = change_cutoff(result_df)

    fper, tper, thresholds = roc_curve(result_df['Actual'], result_df['부도_prob'])
    tmp_proba = [fper, tper]
    
    return tmp_proba

In [ ]:
def Execute_optimal_dt(X_train, y_train, X_test, y_test, hyp): 
    params = hyp
    dt = DecisionTreeClassifier(**params, random_state=0).fit(X_train, y_train)

    best = dt.best_params_

    prediction = dt.predict(X_test)
    prediction_prob =dt.predict_proba(X_test)

    y_test = pd.Series(y_test.values)
    y_test = y_test.to_frame()

    result_df = pd.DataFrame(prediction_prob, columns=['정상_prob','부도_prob'])
    result_df['prediction'] = prediction
    result_df['Actual'] = y_test


    return_df = change_cutoff(result_df)

    fper, tper, thresholds = roc_curve(result_df['Actual'], result_df['부도_prob'])
    tmp_proba = [fper, tper]
    
    return tmp_proba

In [ ]:
def Execute_optimal_knn(X_train, y_train, X_test, y_test, hyp):
    params = hyp
    knn = KNeighborsClassifier(**params).fit(X_train, y_train)

    prediction = knn.predict(X_test)
    prediction_prob =knn.predict_proba(X_test)
    

    y_test = pd.Series(y_test.values)
    y_test = y_test.to_frame()

    result_df = pd.DataFrame(prediction_prob, columns=['정상_prob','부도_prob'])
    result_df['prediction'] = prediction
    result_df['Actual'] = y_test


    return_df = change_cutoff(result_df)
    
    fper, tper, thresholds = roc_curve(result_df['Actual'], result_df['부도_prob'])
    tmp_proba = [fper, tper]

    return tmp_proba

In [ ]:
def Execute_optimal_rf(X_train, y_train, X_test, y_test, hyp):
    params = hyp
    rf= RandomForestClassifier(**params, random_state=0).fit(X_train, y_train)

    prediction = rf.predict(X_test)
    prediction_prob =rf.predict_proba(X_test)

    y_test = pd.Series(y_test.values)
    y_test = y_test.to_frame()

    result_df = pd.DataFrame(prediction_prob, columns=['정상_prob','부도_prob'])
    result_df['prediction'] = prediction
    result_df['Actual'] = y_test


    return_df = change_cutoff(result_df)

    fper, tper, thresholds = roc_curve(result_df['Actual'], result_df['부도_prob'])
    tmp_proba = [fper, tper]

    def plot_feature_importance(model):
        n_features = X_train.shape[1]
        plt.figure(figsize = (10, 10))
        colors = sns.color_palette('cubehelix',len(X_train.columns))
        plt.barh(np.arange(n_features), sorted(model.feature_importances_), align="center", color = colors)
        plt.yticks(np.arange(n_features), X_train.columns)
        plt.xlabel("Random Forest Feature Importance")
        plt.ylabel("Feature")
        plt.ylim(-1, n_features)
        
    plot_feature_importance(rf)
    
    return tmp_proba

# sampling

In [ ]:
def RUSE(X_train, y_train):
  from imblearn.under_sampling import RandomUnderSampler
  rus = RandomUnderSampler()
  
  X_undersampled, y_undersampled = rus.fit_resample(X_train, y_train)
  
  return X_undersampled, y_undersampled

In [ ]:
def ROSE(X_train, y_train):
  from imblearn.over_sampling import RandomOverSampler
  ros = RandomOverSampler()

  X_oversampled, y_oversampled = ros.fit_resample(X_train, y_train)
  
  return X_oversampled, y_oversampled

In [ ]:
def SMOTE(X_train, y_train):
  from imblearn.over_sampling import SMOTE
  smote = SMOTE()
  X_smoted, y_smoted = smote.fit_resample(X_train,y_train)
  
  return X_smoted, y_smoted

# train/test/split

In [ ]:
def split_by_Y(df):
    after_2018  = df.loc[df['회계년도'] >= 2018]['차기부도여부'].value_counts()[1]
    before_2018 = df.loc[df['회계년도'] <  2018]['차기부도여부'].value_counts()[1]
    SIZE = after_2018/len(df)

    test = df[df['회계년도'] >= 2018]
    train = df[df['회계년도'] < 2018]
    
    test.drop(columns = ['회계년도'], inplace=True)
    train.drop(columns= ['회계년도'], inplace=True)

    X_train = train.drop(['차기부도여부'], axis= 1)
    y_train = train['차기부도여부']

    X_test = test.drop(['차기부도여부'], axis=1)
    y_test = test['차기부도여부']

    return X_train, y_train, X_test, y_test

In [ ]:
def split_by_R(df):
    from sklearn.model_selection import train_test_split
    
    data = df.drop(['차기부도여부'], axis= 1)
    target = df['차기부도여부']

    # train_test_split
    X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.178, shuffle=True, stratify=target, random_state=34)
    X_test.drop(columns = ['회계년도'], inplace=True)
    X_train.drop(columns= ['회계년도'], inplace=True)

    return X_train, y_train, X_test, y_test

# main

In [ ]:
# def cal_per(filename, seleted_variables, train_test_split_method , sampling_method, model_name, hyp):
#     df = pd.read_csv(filename+'.csv')
#     global condition

#     df = df[seleted_variables]

#     if train_test_split_method == 'Y' :
#         X_train, y_train, X_test, y_test = split_by_Y(df)
#         print(X_train.head(2))
#     elif train_test_split_method == 'R' :
#         X_train, y_train, X_test, y_test = split_by_R(df)

#     if sampling_method == 'RUSE' :
#         sampled_X_train, sampled_y_train = RUSE( X_train, y_train )
#     elif sampling_method =='ROSE' :
#         sampled_X_train, sampled_y_train = ROSE( X_train, y_train )
#     elif sampling_method =='SMOTE' :
#         sampled_X_train, sampled_y_train = SMOTE( X_train, y_train)
#     elif sampling_method =='None':
#         sampled_X_train, sampled_y_train = X_train, y_train


#     if model_name =='lg':
#         static = Execute_optimal_lg(sampled_X_train, sampled_y_train, X_test, y_test, hyp)
#     elif model_name =='knn':
#         result = Execute_optimal_knn(sampled_X_train, sampled_y_train, X_test, y_test, hyp)
#     elif model_name =='svm':
#         result = Execute_optimal_svm(sampled_X_train, sampled_y_train, X_test, y_test, hyp)
#     elif model_name =='dt':
#         result = Execute_optimal_dt(sampled_X_train, sampled_y_train, X_test, y_test, hyp)
#     elif model_name =='pr':
#         result = Execute_optimal_pr(sampled_X_train, sampled_y_train, X_test, y_test, hyp)
#     elif model_name =='rf':
#         result = Execute_optimal_rf(sampled_X_train, sampled_y_train, X_test, y_test, hyp)

#     return static

In [ ]:
def cal_per(filename, seleted_variables, train_test_split_method , sampling_method, model_name):
    df = pd.read_csv(filename+'.csv')
    global condition

    df = df[seleted_variables]

    if train_test_split_method == 'Y' :
        X_train, y_train, X_test, y_test = split_by_Y(df)
        print(X_train.head(2))
    elif train_test_split_method == 'R' :
        X_train, y_train, X_test, y_test = split_by_R(df)

    if sampling_method == 'RUSE' :
        sampled_X_train, sampled_y_train = RUSE( X_train, y_train )
    elif sampling_method =='ROSE' :
        sampled_X_train, sampled_y_train = ROSE( X_train, y_train )
    elif sampling_method =='SMOTE' :
        sampled_X_train, sampled_y_train = SMOTE( X_train, y_train)
    elif sampling_method =='None':
        sampled_X_train, sampled_y_train = X_train, y_train


    if model_name =='lg':
        result = Execute_optimal_lg_sm(sampled_X_train, sampled_y_train, X_test, y_test)
    elif model_name =='knn':
        result = Execute_optimal_knn(sampled_X_train, sampled_y_train, X_test, y_test, hyp)
    elif model_name =='svm':
        result = Execute_optimal_svm(sampled_X_train, sampled_y_train, X_test, y_test, hyp)
    elif model_name =='dt':
        result = Execute_optimal_dt(sampled_X_train, sampled_y_train, X_test, y_test, hyp)
    elif model_name =='pr':
        result = Execute_optimal_pr(sampled_X_train, sampled_y_train, X_test, y_test, hyp)
    elif model_name =='rf':
        result = Execute_optimal_rf(sampled_X_train, sampled_y_train, X_test, y_test, hyp)

    return result

In [ ]:
target = pd.read_csv('grid_cut_result_auc.csv')
target= target[target['적용모델']!='sgd']

In [ ]:
model_top = pd.read_csv('model_grid_top.csv')
model_top = model_top.fillna(0)
model_top = model_top.sort_values(by = ['적용모델', 'ROC_accuracy'], ascending = False).drop_duplicates(subset = ['적용모델']).reset_index(drop = True)

df_top = pd.read_csv('df_grid_top.csv')
df_top = df_top.fillna(0)
df_top = df_top.sort_values(by = ['데이터프레임', 'ROC_accuracy'], ascending = False).drop_duplicates(subset = ['데이터프레임']).reset_index(drop = True)

In [ ]:
grid_top = pd.read_csv(r'grid_result_auc.csv')

In [ ]:
# def performance_testing(df):
#     result_per = []
#     df_model = []
#     for idx, col in df.iterrows():
#         condition = col[:8].to_list()
#         condition.append(col[14])
#         seleted_variables = condition[7]
#         seleted_variables = re.sub("\[|\]|\'|\'|", "",seleted_variables)
#         seleted_variables = list(seleted_variables.split(', '))
#         seleted_variables.append('차기부도여부')
#         seleted_variables.append('회계년도')
#         df_model.append([condition[0], condition[6]])
#         # print(df_model)
#         if condition[8]==0:
#             tmp = cal_per(condition[0], seleted_variables, condition[1] , condition[5], condition[6], condition[8])
#             result_per.append(tmp)
#         else:
#             condition[8] = literal_eval(condition[8])       
#             tmp = cal_per(condition[0], seleted_variables, condition[1] , condition[5], condition[6], condition[8])
#             result_per.append(tmp)
            
#     def rocvis(fpr , tpr , label ) :
#         plt.plot(fpr, tpr, marker='.', label = label  )

#     fig , ax = plt.subplots(figsize= (15,10))
#     plt.plot([0, 1], [0, 1], linestyle='--')

#     arr = df.index.tolist()
#     for i in arr:
#         rocvis(result_per[i][0] , result_per[i][1], df_model[i])
#     plt.legend(loc='lower right', prop={'size': 15})
#     plt.title('Receiver Operating Characteristic Curve', fontsize = 20)
#     plt.show()

In [ ]:
def performance_testing(df):
    result_per = []
    df_model = []
    for idx, col in df.iterrows():
        condition = col[:8].to_list()
        condition.append(col[14])
        seleted_variables = condition[7]
        seleted_variables = re.sub("\[|\]|\'|\'|", "",seleted_variables)
        seleted_variables = list(seleted_variables.split(', '))
        seleted_variables.append('차기부도여부')
        seleted_variables.append('회계년도')
        df_model.append([condition[0], condition[6]])
        # print(df_model)
        result = cal_per(condition[0], seleted_variables, condition[1] , condition[5], condition[6])
    return result

In [ ]:
grid_top

,데이터프레임,train_test_split(Y/R),t-test(Y/N),변수선택법,VIF(Y/N),sampling 방식,적용모델,선택된 변수,혼동행렬,Accuracy,Precision,Recall,F1-score,ROC_AUC,hyperparameter 변경,cut-off
0,small_business_df,R,t_Y,forward,v_N,RUSE,lg,"['총자본사업이익률', '자기자본구성비율', '종업원수', '단기차입금 대 총차입금...",[[1792 360]\n [ 4 31]],0.834,0.079,0.886,0.146,0.924,{'C': 0.1},N
1,small_business_df,R,t_Y,stepwise,v_N,RUSE,lg,"['자기자본구성비율', '종업원수', '단기차입금 대 총차입금비율', 'CASH F...",[[1768 384]\n [ 5 30]],0.822,0.072,0.857,0.134,0.915,{'C': 0.1},N
2,small_business_df,R,t_Y,stepwise,v_N,SMOTE,lg,"['자기자본구성비율', '종업원수', '단기차입금 대 총차입금비율', 'CASH F...",[[1802 350]\n [ 5 30]],0.838,0.079,0.857,0.145,0.928,{'C': 0.1},N
3,small_business_df,R,t_Y,forward,v_N,ROSE,lg,"['총자본사업이익률', '자기자본구성비율', '종업원수', '단기차입금 대 총차입금...",[[1779 373]\n [ 5 30]],0.827,0.074,0.857,0.137,0.925,{'C': 0.01},N
4,small_business_df,R,t_Y,forward,v_N,SMOTE,lg,"['총자본사업이익률', '자기자본구성비율', '종업원수', '단기차입금 대 총차입금...",[[1799 353]\n [ 5 30]],0.836,0.078,0.857,0.144,0.928,{'C': 0.01},N
5,small_business_df,R,t_Y,stepwise,v_N,ROSE,lg,"['자기자본구성비율', '종업원수', '단기차입금 대 총차입금비율', 'CASH F...",[[1785 367]\n [ 5 30]],0.830,0.076,0.857,0.139,0.926,{'C': 0.01},N
6,small_business_df,R,t_Y,Lasso,v_N,SMOTE,lg,"['자기자본증가율', '종업원1인당 인건비증가율', '총자본순이익률', '자기자본순...",[[1855 297]\n [ 5 30]],0.862,0.092,0.857,0.166,0.919,{'C': 0.1},N
7,small_business_df,R,t_Y,None,v_N,SMOTE,lg,"['자기자본증가율', '종업원1인당 인건비증가율', '총자본사업이익률', '총자본영...",[[1759 393]\n [ 4 31]],0.818,0.073,0.886,0.135,0.916,{'C': 0.001},N
8,small_business_df,R,t_Y,Lasso,v_N,RUSE,lg,"['자기자본증가율', '종업원1인당 인건비증가율', '총자본순이익률', '자기자본순...",[[1754 398]\n [ 4 31]],0.816,0.072,0.886,0.134,0.918,{'C': 0.001},N
9,diff_df,R,t_Y,None,v_N,RUSE,rf,"['growth_rate_수지비율', 'growth_rate_사내유보 대 자기자본비...",[[2289 568]\n [ 2 28]],0.803,0.047,0.933,0.089,0.919,"{'max_depth': 4, 'min_samples_leaf': 4, 'min_s...",N


In [ ]:
grid_top.iloc[5:6]

,데이터프레임,train_test_split(Y/R),t-test(Y/N),변수선택법,VIF(Y/N),sampling 방식,적용모델,선택된 변수,혼동행렬,Accuracy,Precision,Recall,F1-score,ROC_AUC,hyperparameter 변경,cut-off
5,small_business_df,R,t_Y,stepwise,v_N,ROSE,lg,"['자기자본구성비율', '종업원수', '단기차입금 대 총차입금비율', 'CASH F...",[[1785 367]\n [ 5 30]],0.83,0.076,0.857,0.139,0.926,{'C': 0.01},N


In [ ]:
result = performance_testing(grid_top.iloc[5:6])
# performance_testing(model_top)

     pcost       dcost       gap    pres   dres
 0:  0.0000e+00  6.9315e-01  2e+01  1e+00  1e+00
 1:  3.3479e-01  2.4738e-01  9e-01  2e-01  3e-01
 2:  9.1361e-01  8.8031e-01  1e-01  2e-02  1e-01
 3:  9.4520e-01  6.9739e-01  3e-02  4e-02  2e-02
 4:  7.0469e-01  4.2941e-01  1e-03  6e-02  2e-03
 5:  5.0141e-01  4.2428e-01  3e-04  2e-02  6e-04
 6:  4.4457e-01  4.2344e-01  8e-05  4e-03  1e-04
 7:  4.2880e-01  4.2328e-01  2e-05  1e-03  4e-05
 8:  4.2481e-01  4.2326e-01  6e-06  3e-04  1e-05
 9:  4.2370e-01  4.2325e-01  2e-06  9e-05  3e-06
10:  4.2325e-01  4.2325e-01  3e-08  8e-07  1e-08
11:  4.2325e-01  4.2325e-01  3e-10  3e-08  1e-10
Optimal solution found.
                           Logit Regression Results                           
Dep. Variable:                 차기부도여부   No. Observations:                19870
Model:                          Logit   Df Residuals:                    19859
Method:                           MLE   Df Model:                           10
Date:                Tue

In [ ]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 차기부도여부   No. Observations:                19870
Model:                          Logit   Df Residuals:                    19859
Method:                           MLE   Df Model:                           10
Date:                Tue, 21 Dec 2021   Pseudo R-squ.:                  0.4265
Time:                        04:52:18   Log-Likelihood:                -7898.4
converged:                       True   LL-Null:                       -13773.
Covariance Type:            nonrobust   LLR p-value:                     0.000
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
자기자본구성비율            -0.0088      0.001    -13.917      0.000      -0.010      -0.008
종업원수                -0.0053      0.000    -14.084      0.000      -0.006      -0.005
단기차입금 대 총차입금비율       0.0103      0.001     17.873      0.000       0.009       0.011
CASH FLOW 대 부채비율    -0.0025      0.000     -5.750      0.000      -0.003      -0.002
EBITDA(백만원)       -7.62e-05   3.91e-06    -19.469      0.000   -8.39e-05   -6.85e-05
재고자산 대 유동자산비율       -0.0171      0.001    -13.007      0.000      -0.020      -0.015
인건비(백만원)            -0.0002   1.43e-05    -14.069      0.000      -0.000      -0.000
자기자본증가율              0.0006      0.000      4.407      0.000       0.000       0.001
타인자본구성비율             0.0167      0.001     14.660      0.000       0.014       0.019
총자본순이익률             -0.1935      0.015    -13.078      0.000      -0.223      -0.165
기업순이익률               0.1670      0.015     11.220      0.000       0.138       0.196
====================================================================================
"""